# DVHの表示と活用法（応用編）

- **1. 治療計画装置からDVHデータをExportする**
<br><br>
- **2. Pythonでテキストデータを読み込む**
<br><br>
- **3. DVHの表示**
    - **3.1. Structure名とその行番号を調べる**
    - **3.2. すべてのStructureのDVHを表示する**
<br><br>
- **4. データの補間**
    - **4.1. input()関数を使う**

***

## 〇 1. 治療計画装置からDVHデータをExportする

## 〇 2. Pythonでテキストデータを読み込む

下記のようにwith文を用いることで同様の処理ができます。この場合はclose()が不要なのでコードを短くできます。

In [ ]:
with open('DVH1_full.txt',encoding='utf-8') as f:
    data = f.readlines()  

## 〇 3. DVHの表示

### ● 3.1. Structure名とその行番号を調べる

リスト内包表記とzip()関数を用いて，Structure名のリスト（s_name）とその行番号のリスト（s_index）を取得します。

In [ ]:
structure = [(i, j) for i, j in enumerate(data) if 'Structure:' in j]
s_index, s_name = zip(*structure)

print(structure)

### ● 3.2. すべてのStructureのDVHを表示する

すべてのStructureのDVHを表示してみましょう。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

DVH_arrayset = []

for n in range(len(structure)):    
    if s_index[n] != max(s_index):
        DVH_data = data[s_index[n]+18:s_index[n+1]-1]     
    else:
        DVH_data = data[s_index[n]+18:-1]
    DVH_array = np.array([i.split() for i in DVH_data]).astype(float)
    DVH_arrayset.append(DVH_array)
    plt.plot(DVH_arrayset[n][:,0], DVH_arrayset[n][:,2],label=s_name[n].strip().strip('Structure: '))

plt.grid(True)
plt.legend(loc = 'lower left')
plt.xlabel('Dose[%]')
plt.ylabel('Volume[%]')

## 〇 4. 必要な情報の抽出

### ● 4.2. input()関数を使う

input()関数を用いることで，txtファイルを外から渡すことができます。

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

DVH_data = input('File Name >>')
    
with open(DVH_data,encoding='utf-8') as f:    
    data = f.readlines()
        
structure = [(i, j) for i, j in enumerate(data) if 'Structure: ' in j]
s_index, s_name = zip(*structure)
    
DVH_arrayset = []
DxPct_calc = []
VxPct_calc = []
VxGy_calc = []

for n in range(len(structure)):
    if s_index[n] != max(s_index):
        DVH_data = data[s_index[n]+18:s_index[n+1]-1]
    else:
        DVH_data = data[s_index[n]+18:-1]
    DVH_array = np.array([i.split() for i in DVH_data]).astype(float)
    DVH_arrayset.append(DVH_array)
  
    DxPct = interp1d(DVH_arrayset[n][:,2],DVH_arrayset[n][:,0])
    DxPct_calc.append(DxPct)
    VxPct = interp1d(DVH_arrayset[n][:,0],DVH_arrayset[n][:,2])
    VxPct_calc.append(VxPct)
    VxGy = interp1d(DVH_arrayset[n][:,1],DVH_arrayset[n][:,2])
    VxGy_calc.append(VxGy)
    
print('PTV-Rectum')
print('  D95%','%.2f' % DxPct_calc[1](95),'%')
print('  V90%','%.2f' % VxPct_calc[1](90),'%')
print('  Dmax','%.2f' % float(data[s_index[1]+8].strip().split()[3]),'%')
print('  Dmean','%.2f' % float(data[s_index[1]+9].strip().split()[3]),'%')
print('Rectal Wall')
print('  V40Gy','%.2f' % VxGy_calc[0](40),'%')
print('  V60Gy','%.2f' % VxGy_calc[0](60),'%')
print('  V70Gy','%.2f' % VxGy_calc[0](70),'%')
print('  V78Gy','%.2f' % VxGy_calc[0](78),'%')
print('Bladder Wall')
print('  V40Gy','%.2f' % VxGy_calc[4](40),'%')
print('  V70Gy','%.2f' % VxGy_calc[4](70),'%')